In [1]:
import numpy as np

In [2]:
beamType = 'DoppiaRastremazione'

# beamType = 'Curva'
# beamType = 'Centinata'

timberType = 'lamellare'
# timberType = 'massiccio'

In [51]:
# N/mm2
f_mk = 28
f_vk = 3.5
f_c90k = 2.5
f_t90k = 0.5
E0mean = 12600
E005 = 10500
Gmean = 650

gammaM = 1.45
kmod = 0.9

alpha = 3.5 # in gradi °
b = 240
h_ap = 1800-700 # altezza apice
h_0 = 1315-700 # altezza appoggio
l = 16000 

alpha = np.radians(alpha)

In [52]:
# volume della trave 
V_b = (h_ap + h_0) * l/2 * b 
V_b = V_b/10**9 # da mm3 a m3
print('V_b:', V_b, 'm3')

# vale solo per la trave a doppia rastremazione:
# volume della zona di colmo
# formula alternativa: b * h_ap**2 * (1 - 0.25 * np.tan(alpha)) / 10**9
h_ap_05 = h_ap - np.tan(alpha)*0.5*h_ap
V_colmo =  (h_ap_05 + h_ap) * 0.5*h_ap * b
V_colmo = V_colmo/10**9
print('V_colmo:', V_colmo, 'm3')

# volume zona di colmo per la normativa
V = min(2/3 * V_b , V_colmo)
print('2/3 V_b:', 2/3 * V_b, 'm3')
print('V:', V, 'm3')

V_0 = 0.01

V_b: 3.2928 m3
V_colmo: 0.28595959377707486 m3
2/3 V_b: 2.1952 m3
V: 0.28595959377707486 m3


In [53]:
M_d = 641.5 #kNm
V_d = 158.4 #kN

M_d = M_d * 10**6 
V_d = V_d * 10**3 

HP: $ \sigma_{m,\alpha,d} = \sigma_{m,0,d}$

In [54]:
sigma_m_alpha_d = 6 * M_d / (b*h_ap**2) # nel punto in mezzeria
sigma_m_alpha_d

13.25413223140496

# Parte di trave a doppia rastremazione
Capitolo 6.4.3 EC5

## Flessione

In [55]:
f_md = kmod * f_mk / gammaM
f_md

17.379310344827587

da ricontrollare i k

In [56]:
t = 30 # spessore lamelle LVL
r_in = 17000 #raggio interno se curva o centinata. altrimenti non serve
if beamType == 'DoppiaRastremazione':
    k_r = 1
else:
    k_r = 1 if r_in/t >240 else 0.76 + 0.001 * r_in/t
    # t spessore lamelle 
k_r    

1

In [61]:
alpha_ap = np.radians(3.5) # vedi la figura
if beamType == 'DoppiaRastremazione':
    k_l = k_1 = 1 + 1.4*np.tan(alpha_ap) + 5.4*np.tan(alpha_ap)**2
    print('k_l: ', k_l)
else:
    k_1 = 1 + 1.4*np.tan(alpha_ap) + 5.4*np.tan(alpha_ap)**2
    k_2 = 0.35 - 8*np.tan(alpha_ap)
    k_3 = 0.6 + 8.3*np.tan(alpha_ap) - 7.8*np.tan(alpha_ap)**2
    k_4 = 6*np.tan(alpha_ap)**2
    r = r_in + 0.5 * h_ap
    k_l = k_1 + k_2*(h_ap/r) + k_3*(h_ap/r)**2 + k_4*(h_ap/r)**3
    print(k_1)
    print(k_2)
    print(k_3)
    print(k_4)
    print(r)
    print('k_l: ', k_l)

sigma_md = k_l * 6 * M_d / (b*h_ap**2) # nel punto in mezzeria

print('sigma_md: ', sigma_md)
print('f_md: ', f_md)
sigma_md < k_r *f_md


k_l:  1.105828345170509
sigma_md:  14.656795112125653
f_md:  17.379310344827587


True

## Trazione 90 all'apice

In [58]:
k_dis =  1.4 if beamType == 'DoppiaRastremazione' or  beamType == 'Curva' else  1.7
print('k_dis:', k_dis)

k_vol = 1 if timberType == 'massiccio' else (V_0/V)**0.2 
print('k_vol:', k_vol)

k_dis: 1.4
k_vol: 0.5113744974534034


In [62]:
f_t90d = f_t90k * kmod / gammaM
f_t90d

0.3103448275862069

In [63]:
k_dis * k_vol * f_t90d

0.22218340234182352

In [64]:
# solo per trave doppia rastremazione. aggiungere le altre!
k_p = 0.2 * np.tan(alpha) 
k_p

0.012232524030096862

In [65]:
sigma_t90d_ap = 6 * M_d / (b*h_ap**2) # M all'apice!
sigma_t90d_ap

13.25413223140496

In [66]:
k_p * sigma_t90d_ap

0.16213149101874252

In [67]:
print('S:', k_p * sigma_t90d_ap)
print('R:', k_dis * k_vol * f_t90d)

k_p * sigma_t90d_ap < k_dis * k_vol * f_t90d

S: 0.16213149101874252
R: 0.22218340234182352


True

## Taglio + Trazione a $0.5\cdot h_{ap}$ 

In [69]:
print('x di 0.5*h_ap: ', l/2 - 0.5*h_ap) 
print('h trave a 0.5*h_ap: ', h_ap_05)

# Mettere i valori a tale x
M = 638 * 10**6
V = 12.8 * 10**3

x di 0.5*h_ap:  7450.0
h trave a 0.5*h_ap:  1066.3605589172337


In [70]:
f_vd = f_vk * kmod / gammaM
print('f_vd:', f_vd)

b_eff = 2.5/f_vk * b
print('b_eff:', b_eff)

tau_d_05ap = 1.5*V/(b_eff*h_ap_05)
#tau_d_05ap = 1.5*V/(0.67*b*h_ap_05)
tau_d_05ap

f_vd: 2.1724137931034484
b_eff: 171.42857142857144


0.10503014113136656

In [71]:
def sigma_t90d(M_d,b,h,k_p):
    sigma_t90d = 6 * M_d / (b*h**2) * k_p
    print('M_d: ', M_d)
    print('b: ', b)
    print('h: ', h)
    print('k_p: ', k_p)
    print('sigma_t90d: ', sigma_t90d)
    return sigma_t90d

sigma_t90d_05ap = sigma_t90d(M , b , h_ap_05 , k_p)

M_d:  638000000
b:  240
h:  1066.3605589172337
k_p:  0.012232524030096862
sigma_t90d:  0.1715807720839749


In [72]:
tau_d_05ap/f_vd + sigma_t90d_05ap/(f_t90d * k_dis * k_vol) 

0.8205955858537919

In [73]:
print('k_dis: ', k_dis)
print('k_vol: ', k_vol)

tau_d_05ap/f_vd + sigma_t90d_05ap/(f_t90d * k_dis * k_vol) < 1

k_dis:  1.4
k_vol:  0.5113744974534034


True

# Parte di trave a rastremazione semplice
Capitolo 6.4.2 EC5. Fino a $0.5 h_{ap}$ dall'asse di simmetria

In [75]:
M_max = 515*10**6 # M corrispondente alla x_max

Q = 19.5595 #kN/m

In [76]:
# Solo se trave simmetrica e carico distribuito!
x_max = l * h_0 / (2*h_ap)
print('x_max: ', x_max)

h_max = h_ap - np.tan(alpha)*(l/2 -x_max) # altezza corrispondente alla x_max
print('h_max: ', h_max)

#sigma_md_max = 6*M_max / (b*h_max**2)
sigma_md_max = 0.75*Q*l**2/b * 1/(h_0*(2*h_ap-h_0))
print('sigma_md_max: ', sigma_md_max)

x_max:  4472.727272727273
h_max:  884.2627580146553
sigma_md_max:  16.052524941653203


In [77]:
f_c90d = f_c90k * kmod / gammaM
f_c90d

1.5517241379310345

### Bordo inclinato (compresso)

calcolare l'altro k_m se il momento va dall'altra parte e quindi è teso

In [78]:
k_mAlpha_compressione = 1 / np.sqrt(1 + (np.tan(alpha)*f_md/(1.5*f_vd))**2 + ((np.tan(alpha))**2*f_md/f_c90d)**2 )
k_mAlpha_compressione

0.9499446894142974

In [79]:
k_mAlpha_compressione * f_md

16.50938356775193

In [80]:
sigma_md_max < k_mAlpha_compressione * f_md

True

### Bordo non inclinato (teso)

In [81]:
print('sigma_md_max: ', sigma_md_max)
print('f_md: ', f_md)

sigma_md_max < f_md

sigma_md_max:  16.052524941653203
f_md:  17.379310344827587


True

# Freccia

In [82]:
q = [8.6428,10.9167]

k_m = (h_0/h_ap)**3 * 1/(0.15+0.85*(h_0/h_ap))
print('k_m: ', k_m)
k_v = 2/(1 + (h_ap/h_0)**(2/3))
print('k_k_vm: ', k_v)

#Nella sezone di appoggio e poi vengono usati i due coefficienti
J_0 = b*h_0**3 / 12
print('J_0:', J_0/10**9, '* 10**9 mm4')
print('\n')

chi = 1.2
def w_inst(q):
    w_inst_M = k_m * 5/384 * q * l**4 / (E0mean * J_0) 
    w_inst_V = chi * 1/8 * q * l**2 / (Gmean * b * h_0)
    w_inst = w_inst_M + w_inst_V
    return w_inst

w=[]
for qq in q:
    w_i = w_inst(qq)
    w.append(w_i)
    print(w_i)
    print(l/w_i)
    print('\n')

print('w_inst totale = ',np.sum(w))
print(l/25)
print(l/300)
print(l/500)

k_m:  0.2795177411069153
k_k_vm:  0.808576500722525
J_0: 4.6521675 * 10**9 mm4


38.628010283689605
414.2072004872558


48.79094736242356
327.92968501206906


w_inst totale =  87.41895764611317
53.333333333333336
32.0


In [85]:
l/w_inst(Q)

183.0266618457146

In [86]:
k_def = 0.8
psi2 = 0.

w_netFin_1 = w[0]*(1 + k_def) 
print('w_netFin_1: ', w_netFin_1)

w_netFin_2 = w[1]*(1 + k_def*psi2)
print('w_netFin_2: ', w_netFin_2)

w_netFin_tot = w_netFin_1+w_netFin_2
print('w_netFin_tot: ', w_netFin_tot)
print('l/w_netFin_tot: ', l/w_netFin_tot)

w_netFin_1:  69.5304185106413
w_netFin_2:  48.79094736242356
w_netFin_tot:  118.32136587306485
l/w_netFin_tot:  135.22494337298977
